# PGMW-3070

Updating the dataset: B010020100000
* Detector images reveal a star in the LWRS aperture, so I changed the background regions and re-extracted the spectra.  
* LiF 1 is significantly brighter than the LiF 2 and SiC spectra, but a reasonable match to the star's STIS spectrum. The LiF 2B spectrum droops below zero at short wavelengths. Because LiF 1B is is much brighter than LiF 2A, we use the latter only between 1087.5 and 1095 A.


COMMENT File updated 26 January 2023 <br>
COMMENT Star in LWRS aperture caused background to be overestimated. <br>
COMMENT We have re-extracted all spectra using new background regions.


In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Function to read a regular FUSE spectral file.

def read_fuse(filename):
    f = fits.open(filename)
    data = f[1].data
    f.close()
    spec = make_spectrum (data)
    return data, spec

In [ ]:
# Read keywords from file header.

filename = 'b010020100000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
f.close() 
    
# Target is in MDRS aperture.  LiF 1A is the guide channel.

In [ ]:
# A star in the LWRS aperture was included in the background calculation. 
# I have re-defined the background regions and re-extracted all of the spectra.
# Let's look at the results.

lif1a_data, lif1a = read_fuse('B01002010001alif2ttagfcal.fit')
lif1b_data, lif1b = read_fuse('B01002010001blif2ttagfcal.fit')
lif2a_data, lif2a = read_fuse('B01002010002alif2ttagfcal.fit')
lif2b_data, lif2b = read_fuse('B01002010002blif2ttagfcal.fit')
sic1a_data, sic1a = read_fuse('B01002010001asic2ttagfcal.fit')
sic1b_data, sic1b = read_fuse('B01002010001bsic2ttagfcal.fit')
sic2a_data, sic2a = read_fuse('B01002010002asic2ttagfcal.fit')
sic2b_data, sic2b = read_fuse('B01002010002bsic2ttagfcal.fit')

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed, combined spectra.  

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()

# The LiF 2B channel has problems below 1000 A, but the others look OK.

In [ ]:
# Rescale the other channels to match LiF 1A.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes.

mean = sub_lif1a.mean()
scale_lif2b = mean/sub_lif2b.mean()
print ('Scale LiF 2B by', scale_lif2b)
scale_sic1a = mean/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1a)
scale_sic2b = mean/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic2b)

In [ ]:
# Scale the spectra.

lif2b *= scale_lif2b
sic1a *= scale_sic1a
sic2b *= scale_sic2b

lif2b_bsmooth *= scale_lif2b
sic1a_bsmooth *= scale_sic1a
sic2b_bsmooth *= scale_sic2b

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1030,1080])
ax.set_ylim([-2, 13])

# This looks pretty good.

In [ ]:
# What's going on at long wavelengths?

f, ax = plt.subplots()
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A')
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B')
ax.legend()

# We'll scale LiF 2A to match LiF 1B.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1100*u.AA, 1120*u.AA)
sub_lif1b = extract_region(lif1b, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes.

scale_lif2a = sub_lif1b.mean()/sub_lif2a.mean()
print ('Scale LiF 2A by', scale_lif2a)

In [ ]:
# Scale the spectrum.

lif2a_bsmooth *= scale_lif2a

In [ ]:
# Plot the rescaled spectrum.

f, ax = plt.subplots()  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.set_xlim([1090,1120])
ax.legend()

# The match is not great, but the S/N for LiF 2A is low.
# We'll use LiF 2A only when necessary.

In [ ]:
# Which has the higher S/N, LiF 2A or SiC 2B?

f, ax = plt.subplots()  
ax.step(lif2a.spectral_axis, lif2a_data['flux']/lif2a_data['error'], label='LiF 2A') 
ax.step(sic2b.spectral_axis, sic2b_data['flux']/sic2b_data['error'], label='SiC 2B') 
ax.set_xlim([1090,1105])
ax.set_ylim([0, 6])
ax.legend()

# Looks like LiF 2A has the higher S/N.  Use it instead of LiF 2A.

In [ ]:
# Compare with a STIS spectrum of the same star.

stis = Spectrum1D.read("oeuf2s030_x1d.fits", format="HST/STIS")
print (fits.getval(filename, 'TARGNAME', 0))
print (fits.getval(filename, 'APERTURE', 0))
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS', color='g')  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1150,1190])
ax.set_ylim([0, 12])

# The scaling looks fine, though the difference in the broad absorption feature at 1177 A is troubling.

In [ ]:
# At short wavelengths, we must boot-strap our way down.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([980, 1010])

In [ ]:
# First, we use LiF 1A to rescale SiC 2A.

# Select a spectral region.

region = SpectralRegion(995*u.AA, 1005*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_sic2a = extract_region(sic2a, region)

# Compute ratio of the flux to LiF 1A.

scale_sic2a = sub_lif1a.mean()/sub_sic2a.mean()
print ('Scale SiC 2A by', scale_sic2a)

# Scale the spectrum.

sic2a *= scale_sic2a
sic2a_bsmooth *= scale_sic2a

In [ ]:
# Then use SiC 2A to rescale SiC 1B.

# Select a spectral region.

region = SpectralRegion(957*u.AA, 970*u.AA)
sub_sic1b = extract_region(sic1b, region)
sub_sic2a = extract_region(sic2a, region)

# Compute ratio of the flux to SiC 2A.

scale_sic1b = sub_sic2a.mean()/sub_sic1b.mean()
print ('Scale SiC 1B by', scale_sic1b)

# Scale the spectrum.

sic1b *= scale_sic1b
sic1b_bsmooth *= scale_sic1b

# plot the scaled spectra.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A')
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([980, 1010])

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A.

shift_lif2 = compute_shift(lif2b_data, lif1a_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif1a_data, 1040, 1070)

print ('Shift LiF 2 by ', shift_lif2, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif2 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
lif2a_shift = Spectrum1D(spectral_axis=lif2a.spectral_axis + shift_lif2 * u.AA, flux=lif2a.flux)
lif2b_shift = Spectrum1D(spectral_axis=lif2b.spectral_axis + shift_lif2 * u.AA, flux=lif2b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([-2,12])

In [ ]:
# We assemble the spectrum.

filename = 'b010020100000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 917.5))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 917.5))
data['flux'][g] = float(scale_sic1b) * sic1b_data['flux'][h]
data['error'][g] = float(scale_sic1b) * sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 998))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 998))
data['flux'][g] = float(scale_sic2a) * sic2a_data['flux'][h]
data['error'][g] = float(scale_sic2a) * sic2a_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 998) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 998) & (lif1a_wave < 1082.5))
data['flux'][g] = lif1a_data['flux'][h]
data['error'][g] = lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1087.5))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1087.5))
data['flux'][g] = float(scale_sic1a) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087.5) & (data['wave'] < 1095))
h = np.where((lif2a_wave > 1087.5) & (lif2a_wave < 1095))
data['flux'][g] = float(scale_lif2a) * lif2a_data['flux'][h]
data['error'][g] = float(scale_lif2a) * lif2a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1095) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1095) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = lif1b_data['flux'][h]
data['error'][g] = lif1b_data['error'][h]

hdr['comment'] = ''
hdr['comment'] = 'File updated 26 January 2023.'
hdr['comment'] = 'Star in LWRS aperture caused background to be overestimated.'
hdr['comment'] = 'We have re-extracted all spectra using new background regions.'

f.writeto('level0_b010020100000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'b010020100000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_b010020100000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 

ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 

ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([-2e-13,1.5e-12])

In [ ]:
# Let's look more closely at the region sampled only by the SiC channels.

f, ax = plt.subplots()  
ax.step(new['wave'], new['flux'], label='NEW FLUX')
ax.step(old['wave'], old['flux'], label='OLD FLUX') 
ax.legend()
ax.set_xlim([1075,1100])
ax.set_ylim([-2e-13,1.5e-12])

# Seems reasonable.